In [1]:
import os
from pathlib import Path
import json

In [2]:
import networkx

In [3]:
graph =networkx.complete_graph(4)
networkx.draw(graph)

In [22]:


tcabs_ = []
tcabs_

In [102]:
## Réalisé hors-dojo pour anonymiser le code.

words = []
with open('/usr/share/dict/french') as wordfile:
    words = wordfile.read().split('\n')

words = [word for word in words if "'" not in word and len(word)> 3 and "-" not in word]

import random
random.shuffle(words)

dict_ = {}

def anonymize(tcab):
    bits = tcab.get('nom').split('-')
    sub_bits = [ sub_bit.strip() for sub_bit in bits[2].split('//')]

    if tcab.get('type_nom') not in dict_:
        replacement = words.pop()
        dict_[tcab.get('type_nom')] = replacement
    
    for bit in bits:
        if bit not in dict_:
            replacement = words.pop()
            dict_[bit] = replacement
    for bit in sub_bits:
        if bit and bit not in dict_:
            replacement = words.pop()
            dict_[bit] = replacement

    bits_2_replacement = bits[2]
    if sub_bits[0]:
         bits_2_replacement = bits_2_replacement.replace(sub_bits[0], dict_.get(sub_bits[0]))
    if sub_bits[1]:
        bits_2_replacement = bits_2_replacement.replace(sub_bits[1], dict_.get(sub_bits[1]))
    if sub_bits[2]:
        bits_2_replacement =  bits_2_replacement.replace(sub_bits[2], dict_.get(sub_bits[2]))

    def nth(list_, position):
        return (list_[position:] or [None])[0]
    
    return {'type_nom': dict_.get(tcab.get('type_nom')), 'nom': '-'.join(
        [ bit for bit in [
            'cable',
            dict_.get(bits[1]),
            bits_2_replacement,
            dict_.get(nth(bits, 3)),
            dict_.get(nth(bits, 4)),
            dict_.get(nth(bits, 5)),
            dict_.get(nth(bits, 6)),
            dict_.get(nth(bits, 7)),
            dict_.get(nth(bits, 8)),
            dict_.get(nth(bits, 9)),
            dict_.get(nth(bits, 10))
        ] if bit])    
    }

tcabs = [ anonymize(tcab) for tcab in tcabs_ ]

#with open('/home/user/anonymized.json', 'w+') as anon:
#    anon.write((json.dumps(tcabs)))
with open('/home/user/anonymized.json') as anon:
    tcabs = json.loads(anon.read())
tcabs


In [78]:
ma_nouvelle_liste = [ tcab.get('nom') for tcab in tcabs ]
ma_nouvelle_liste

In [79]:
noms = ma_nouvelle_liste
nombres_de_champs = [len(nom.split('-')) for nom in noms]

In [81]:
import functools
acc_ = {}
def accumule(acc, valeur):
    v = acc.get(valeur, 0)
    acc[valeur] = v+1
    return acc
    

stats = functools.reduce(accumule, nombres_de_champs, acc_)

In [82]:
stats

In [83]:
[ nom for nom in noms if len(nom.split('-')) == 10]

In [86]:
copper_only = [tcab.get('nom') for tcab in tcabs if tcab.get('type_nom') == 'inférassions']

functools.reduce(accumule, [len(cable.split('-')) for cable in copper_only],   {})

In [87]:
[nom for nom in copper_only if len(nom.split('-')) == 3 ]

In [88]:
ends = [(cable.split('-')[2].split('//')[1].strip(),cable.split('-')[2].split('//')[2].strip())  for cable in copper_only]
ends

In [14]:
!pip install graphviz

In [89]:
import graphviz

In [90]:
my_graph = graphviz.Graph(engine = 'twopi')

In [91]:
for end in ends:
    my_graph.edge(end[0],end[1])

In [18]:
my_graph.format= 'svg'
my_graph.render()

In [92]:
my_graph.view()

In [20]:
ends_all = [(cable.split('-')[2].split('//')[1].strip(),cable.split('-')[2].split('//')[2].strip())  for cable in noms]
ends_all

In [75]:
graph_of_all = graphviz.Graph(engine = 'twopi')
for end in ends_all:
    if end[0] != end[1]:
        graph_of_all.edge(end[0],end[1], color='blue')

In [76]:
graph_of_all.format='svg'
graph_of_all.view()